In [1]:
! pip install --quiet git+https://github.com/expectedparrot/edsl.git


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from edsl import Scenario

In [ ]:
from occupations import get_scenarios
scenarios = get_scenarios(1)

In [3]:
#options = ["Slice bread", "Spread mayo", "Add meats and cheese", "Wrap sandwich", "Take payment"]
#scenarios = [Scenario({"occupation": "Sandwich artist", "task": task}) for task in options]

In [4]:
from question_bank import task_survey
results = task_survey.by(scenarios).run()
columns = ["task"] +  task_survey.question_names
results.select(*columns).print() 
results.select(*columns).to_csv("task_survey.csv")

┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ scenario        ┃ answer     ┃ answer       ┃ answer          ┃ answer        ┃ answer        ┃ answer          ┃
┃ .task           ┃ .info_good ┃ .gen_ai_help ┃ .trigger        ┃ .easy_to_tell ┃ .manager_task ┃ .ratio_describ… ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Slice bread     │ No         │ 0            │ External        │ 1             │ Almost none   │ 0.1             │
│                 │            │              │ trigger like a  │               │               │                 │
│                 │            │              │ customer        │               │               │                 │
│                 │            │              │ request         │               │               │                 │
├─────────────────┼────────────┼──────────────┼─────────────────┼───────────────┼───────────────┼─────────────────┤
│ Add meats and   │ No         │ 0            │ External        │ 1             │ Almost none   │ 0.1             │
│ cheese          │            │              │ trigger like a  │               │               │                 │
│                 │            │              │ customer        │               │               │                 │
│                 │            │              │ request         │               │               │                 │
├─────────────────┼────────────┼──────────────┼─────────────────┼───────────────┼───────────────┼─────────────────┤
│ Wrap sandwich   │ No         │ 0            │ Completion of a │ 1             │ Almost none   │ 0.1             │
│                 │            │              │ previous step   │               │               │                 │
│                 │            │              │ in some         │               │               │                 │
│                 │            │              │ productive      │               │               │                 │
│                 │            │              │ process         │               │               │                 │
├─────────────────┼────────────┼──────────────┼─────────────────┼───────────────┼───────────────┼─────────────────┤
│ Take payment    │ No         │ 0            │ External        │ 1             │ Almost none   │ 0.1             │
│                 │            │              │ trigger like a  │               │               │                 │
│                 │            │              │ customer        │               │               │                 │
│                 │            │              │ request         │               │               │                 │
├─────────────────┼────────────┼──────────────┼─────────────────┼───────────────┼───────────────┼─────────────────┤
│ Spread mayo     │ No         │ 0            │ External        │ 1             │ Almost none   │ 0.1             │
│                 │            │              │ trigger like a  │               │               │                 │
│                 │            │              │ customer        │               │               │                 │
│                 │            │              │ request         │               │               │                 │
└─────────────────┴────────────┴──────────────┴─────────────────┴───────────────┴───────────────┴─────────────────┘

In [5]:
task_survey.show_questions()

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Question Name ┃ Question Type   ┃ Question Text                                            ┃ Options            ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ info_good     │ multiple_choice │ Consider this occupation {{ occupation }} and this task: │ Yes, Partially, No │
│               │                 │ {{ task }}.                                              │                    │
│               │                 │ Is this task output an information good (i.e.,           │                    │
│               │                 │ writing/text, audio, a decision, an image, a video)?     │                    │
│               │                 │                                                          │                    │
└───────────────┴─────────────────┴──────────────────────────────────────────────────────────┴────────────────────┘

Question:info_good,┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━
━━━━━━━━━━━━━━━━━━┓
┃ Question Name ┃ Question Type   ┃ Question Text                                            ┃ Options            ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ info_good     │ multiple_choice │ Consider this occupation {{ occupation }} and this task: │ Yes, Partially, No │
│               │                 │ {{ task }}.                                              │                    │
│               │                 │ Is this task output an information good (i.e.,           │                    │
│               │                 │ writing/text, audio, a decision, an image, a video)?     │                    │
│               │                 │                                                          │                    │
└───────────────┴─────────────────┴──────────────────────────────────────────────────────────┴────────────────────┘

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Question Name ┃ Question Type ┃ Question Text                                ┃ Options                          ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gen_ai_help   │ linear_scale  │ Consider this occupation: {{ occupation}}    │ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 │
│               │               │ and this task: {{ task }}.                   │                                  │
│               │               │ How helpful would generative AI to be        │                                  │
│               │               │ someone performing the task?                 │                                  │
│               │               │                                              │                                  │
└───────────────┴───────────────┴──────────────────────────────────────────────┴──────────────────────────────────┘

Question:gen_ai_help,┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━
━━━━━━━━━━━━━━━━━━━━┓
┃ Question Name ┃ Question Type ┃ Question Text                                ┃ Options                          ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gen_ai_help   │ linear_scale  │ Consider this occupation: {{ occupation}}    │ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 │
│               │               │ and this task: {{ task }}.                   │                                  │
│               │               │ How helpful would generative AI to be        │                                  │
│               │               │ someone performing the task?                 │                                  │
│               │               │                                              │                                  │
└───────────────┴───────────────┴──────────────────────────────────────────────┴──────────────────────────────────┘

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Question Name ┃ Question Type   ┃ Question Text                         ┃ Options                               ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ trigger       │ multiple_choice │ Consider the following task {{ task   │ Completion of a previous step in some │
│               │                 │ }} as part of the occupation {{       │ productive process, External trigger  │
│               │                 │ occupation }}.                        │ like a customer request, External     │
│               │                 │     What best describes when this     │ trigger a request from another part   │
│               │                 │ task is done?                         │ of the organization, Internal choice  │
│               │                 │                                       │ - can be scheduled whenever           │
│               │                 │                                       │ convenient                            │
└───────────────┴─────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘

...

Question:easy_to_tell,┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━
━━━━━━━━━━━━━━━━━━━━━┓
┃ Question Name ┃ Question Type ┃ Question Text                                ┃ Options                          ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ easy_to_tell  │ linear_scale  │ Consider the occupation {{ occupation }}.    │ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 │
│               │               │ And consider the task done by someone in     │                                  │
│               │               │ this occupation: {{ task }}.                 │                                  │
│               │               │ Rate how easy it is for a non-expert to tell │                                  │
│               │               │ if the task is done correctly, on a scale of │                                  │
│               │               │ 1 to 10.                                     │                                  │
│               │               │                                              │                                  │
│               │               │ The task: {{ task }}.                        │                                  │
│               │               │ 1 = Very easy for a non-expert to tell if    │                                  │
│               │               │ done correctly.                              │                                  │
│               │               │ 10 = only an expert would know if done       │                                  │
│               │               │ correctly.                                   │                                  │
│               │               │                                              │                                  │
└───────────────┴───────────────┴──────────────────────────────────────────────┴──────────────────────────────────┘

┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Question Name ┃ Question Type   ┃ Question Text                         ┃ Options                               ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ manager_task  │ multiple_choice │                                       │ None, Almost none, Some - less than   │
│               │                 │     Consider this occupation: {{      │ an hour, Great deal - mulitple hours  │
│               │                 │ occupation }}.                        │                                       │
│               │                 │     Part of this occupation is this   │                                       │
│               │                 │ task: {{ task }}.                     │                                       │
│               │                 │                                       │                                       │
│               │                 │     Imagine a manager who was in this │                                       │
│               │                 │ occupation is now asking a worker     │                                       │
│               │                 │ with the same occupation to do the    │                                       │
│               │                 │ task.                                 │                                       │
│               │                 │     How much specific instruction is  │                                       │
│               │                 │ this task likely to require?          │                                       │
│               │                 │                                       │                                       │
└───────────────┴─────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘

Question:manager_task,┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━
━━━━━━━━━━━━━━━━━━━━━┓
┃ Question Name ┃ Question Type   ┃ Question Text                         ┃ Options                               ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ manager_task  │ multiple_choice │                                       │ None, Almost none, Some - less than   │
│               │                 │     Consider this occupation: {{      │ an hour, Great deal - mulitple hours  │
│               │                 │ occupation }}.                        │                                       │
│               │                 │     Part of this occupation is this   │                                       │
│               │                 │ task: {{ task }}.                     │                                       │
│               │                 │                                       │                                       │
│               │                 │     Imagine a manager who was in this │                                       │
│               │                 │ occupation is now asking a worker     │                                       │
│               │                 │ with the same occupation to do the    │                                       │
│               │                 │ task.                                 │                                       │
│               │                 │     How much specific instruction is  │                                       │
│               │                 │ this task likely to require?          │                                       │
│               │                 │                                       │                                       │
└───────────────┴─────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘

┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Question Name        ┃ Question Type ┃ Question Text                                                  ┃ Options ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ratio_describe_to_do │ numerical     │                                                                │ None    │
│                      │               │     Consider this occupation: {{ occupation }}.                │         │
│                      │               │     Part of this occupation is this task: {{ task }}.          │         │
│                      │               │                                                                │         │
│                      │               │     Imagine a manager who was in this occupation is now asking │         │
│                      │               │ a worker with the same occupation to do the task.              │         │
│                      │               │     The worker is expert and does not need instruction in how  │         │
│                      │               │ to do the task generally.                                      │         │
│                      │               │     They only need to be instructed on any specifics.          │         │
│                      │               │                                                                │         │
│                      │               │     What is the ratio of time to describe this task to time to │         │
│                      │               │ do it on their own?                                            │         │
│                      │               │     E.g., if it takes just as long to describe the task as to  │         │
│                      │               │ do it on their own, then 1.                                    │         │
│                      │               │     If the task could be described in 5 minutes but the task   │         │
│                      │               │ takes 10 minutes, then 0.5.                                    │         │
│                      │               │                                                                │         │
└──────────────────────┴───────────────┴────────────────────────────────────────────────────────────────┴─────────┘

Question:ratio_describe_to_do,┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Question Name        ┃ Question Type ┃ Question Text                                                  ┃ Options ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ratio_describe_to_do │ numerical     │                                                                │ None    │
│                      │               │     Consider this occupation: {{ occupation }}.                │         │
│                      │               │     Part of this occupation is this task: {{ task }}.          │         │
│                      │               │                                                                │         │
│                      │               │     Imagine a manager who was in this occupation is now asking │         │
│                      │               │ a worker with the same occupation to do the task.              │         │
│                      │               │     The worker is expert and does not need instruction in how  │         │
│                      │               │ to do the task generally.                                      │         │
│                      │               │     They only need to be instructed on any specifics.          │         │
│                      │               │                                                                │         │
│                      │               │     What is the ratio of time to describe this task to time to │         │
│                      │               │ do it on their own?                                            │         │
│                      │               │     E.g., if it takes just as long to describe the task as to  │         │
│                      │               │ do it on their own, then 1.                                    │         │
│                      │               │     If the task could be described in 5 minutes but the task   │         │
│                      │               │ takes 10 minutes, then 0.5.                                    │         │
│                      │               │                                                                │         │
└──────────────────────┴───────────────┴────────────────────────────────────────────────────────────────┴─────────┘